In [3]:
# needed to see what libaraies I was using.
!pip freeze

anyio==3.6.2
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arrow==1.2.3
asttokens==2.2.1
attrs==23.1.0
backcall==0.2.0
beautifulsoup4==4.12.2
bleach==6.0.0
cffi==1.15.1
comm==0.1.3
contourpy==1.0.7
cycler==0.11.0
debugpy==1.6.7
decorator==5.1.1
defusedxml==0.7.1
executing==1.2.0
fastjsonschema==2.17.1
fonttools==4.39.4
fqdn==1.5.1
idna==3.4
ipykernel==6.23.1
ipython==8.13.2
ipython-genutils==0.2.0
ipywidgets==8.0.6
isoduration==20.11.0
jedi==0.18.2
Jinja2==3.1.2
joblib==1.2.0
jsonpointer==2.3
jsonschema==4.17.3
jupyter==1.0.0
jupyter-console==6.6.3
jupyter-events==0.6.3
jupyter_client==8.2.0
jupyter_core==5.3.0
jupyter_server==2.5.0
jupyter_server_terminals==0.4.4
jupyterlab-pygments==0.2.2
jupyterlab-widgets==3.0.7
kiwisolver==1.4.4
MarkupSafe==2.1.2
matplotlib==3.7.1
matplotlib-inline==0.1.6
mistune==2.0.5
nbclassic==1.0.0
nbclient==0.8.0
nbconvert==7.4.0
nbformat==5.8.0
nest-asyncio==1.5.6
notebook==6.5.4
notebook_shim==0.2.3
numpy==1.24.3
packaging==23.1
pandas==2.0.1
pandocfilt

# Libraries

In [1]:
# copied these from the other note book present in the tree.
 
from collections import defaultdict

import pandas as pd
import numpy as np
import scipy
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
import seaborn as sns

import surprise as sp
import time

from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

# Call in Dataset

In [4]:
UsersDF = pd.read_csv('../raw_data/users_cleaned.csv')

In [5]:
UsersDF.head()


,username,user_id,user_watching,user_completed,user_onhold,user_dropped,user_plantowatch,user_days_spent_watching,gender,location,birth_date,access_rank,join_date,last_online,stats_mean_score,stats_rewatched,stats_episodes
0,karthiga,2255153,3,49,1,0,0,55.091667,Female,"Chennai, India",1990-04-29 00:00:00,NaN,2013-03-03 00:00:00,2014-02-04 01:32:00,7.43,0.0,3391
1,Damonashu,37326,45,195,27,25,59,82.574306,Male,"Detroit,Michigan",1991-08-01 00:00:00,NaN,2008-02-13 00:00:00,2017-07-10 06:52:54,6.15,6.0,4903
2,bskai,228342,25,414,2,5,11,159.483333,Male,"Nayarit, Mexico",1990-12-14 00:00:00,NaN,2009-08-31 00:00:00,2014-05-12 16:35:00,8.27,1.0,9701
3,terune_uzumaki,327311,5,5,0,0,0,11.394444,Female,"Malaysia, Kuantan",1998-08-24 00:00:00,NaN,2010-05-10 00:00:00,2012-10-18 19:06:00,9.70,6.0,697
4,Bas_G,5015094,35,114,6,20,175,30.458333,Male,"Nijmegen, Nederland",1999-10-24 00:00:00,NaN,2015-11-26 00:00:00,2018-05-10 20:53:37,7.86,0.0,1847


# Understanding the Data


## Exploratory Data Analysis


In [ ]:
# the data's structure, distribution, and relationships

In [16]:
UsersDF.shape

(108711, 17)

In [7]:
# find the basic overview
UsersDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108711 entries, 0 to 108710
Data columns (total 17 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   username                  108710 non-null  object 
 1   user_id                   108711 non-null  int64  
 2   user_watching             108711 non-null  int64  
 3   user_completed            108711 non-null  int64  
 4   user_onhold               108711 non-null  int64  
 5   user_dropped              108711 non-null  int64  
 6   user_plantowatch          108711 non-null  int64  
 7   user_days_spent_watching  108711 non-null  float64
 8   gender                    108711 non-null  object 
 9   location                  108706 non-null  object 
 10  birth_date                108711 non-null  object 
 11  access_rank               0 non-null       float64
 12  join_date                 108711 non-null  object 
 13  last_online               108711 non-null  o

In [8]:
UsersDF.describe() 

,user_id,user_watching,user_completed,user_onhold,user_dropped,user_plantowatch,user_days_spent_watching,access_rank,stats_mean_score,stats_rewatched,stats_episodes
count,1.087110e+05,108711.000000,108711.000000,108711.000000,108711.000000,108711.000000,108711.000000,0.0,108711.000000,108711.000000,108711.000000
mean,1.817687e+06,14.767503,196.458178,11.388167,11.733716,75.578589,61.913873,NaN,7.747612,14.197993,3779.871255
std,1.998269e+06,32.746591,244.945751,30.830825,30.978991,178.653664,59.211762,NaN,1.451368,55.376799,3678.336729
min,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.500694,NaN,0.000000,0.000000,7.000000
25%,1.994345e+05,3.000000,50.000000,0.000000,0.000000,6.000000,21.066319,NaN,7.330000,0.000000,1269.500000
50%,5.675030e+05,7.000000,123.000000,4.000000,3.000000,27.000000,46.190278,NaN,7.890000,1.000000,2797.000000
75%,3.477036e+06,16.000000,254.000000,12.000000,12.000000,81.000000,84.461806,NaN,8.460000,10.000000,5135.000000
max,7.251737e+06,2934.000000,5479.000000,2562.000000,2457.000000,12051.000000,952.654595,NaN,10.000000,9404.000000,62347.000000


In [45]:
# checking to see what the unique values held in the DataFrame
UsersDF.nunique()

username                    108705
user_id                     108705
user_watching                  415
user_completed                1809
user_onhold                    374
user_dropped                   398
user_plantowatch              1347
user_days_spent_watching     85430
gender                           3
location                     40438
birth_date                    9347
join_date                     4247
last_online                 106546
stats_mean_score               661
stats_rewatched                563
stats_episodes               14290
dtype: int64

In [50]:
# what types are they and how do I need to encode them?
UsersDF.dtypes
# Gender needs to be one hot encoded
# dates need to be changed to Data Types

username                     object
user_id                       int64
user_watching                 int64
user_completed                int64
user_onhold                   int64
user_dropped                  int64
user_plantowatch              int64
user_days_spent_watching    float64
gender                       object
location                     object
birth_date                   object
join_date                    object
last_online                  object
stats_mean_score            float64
stats_rewatched             float64
stats_episodes                int64
dtype: object

# Handing Missing Values

In [30]:
# these two columns were not useful due to too much missing data. 
UsersDF.drop('access_rank', inplace= True, axis = 1)
UsersDF.drop('encoded_gender', inplace= True, axis = 1)
UsersDF

,username,user_id,user_watching,user_completed,user_onhold,user_dropped,user_plantowatch,user_days_spent_watching,gender,location,birth_date,join_date,last_online,stats_mean_score,stats_rewatched,stats_episodes
0,karthiga,2255153,3,49,1,0,0,55.091667,Female,"Chennai, India",1990-04-29 00:00:00,2013-03-03 00:00:00,2014-02-04 01:32:00,7.43,0.0,3391
1,Damonashu,37326,45,195,27,25,59,82.574306,Male,"Detroit,Michigan",1991-08-01 00:00:00,2008-02-13 00:00:00,2017-07-10 06:52:54,6.15,6.0,4903
2,bskai,228342,25,414,2,5,11,159.483333,Male,"Nayarit, Mexico",1990-12-14 00:00:00,2009-08-31 00:00:00,2014-05-12 16:35:00,8.27,1.0,9701
3,terune_uzumaki,327311,5,5,0,0,0,11.394444,Female,"Malaysia, Kuantan",1998-08-24 00:00:00,2010-05-10 00:00:00,2012-10-18 19:06:00,9.70,6.0,697
4,Bas_G,5015094,35,114,6,20,175,30.458333,Male,"Nijmegen, Nederland",1999-10-24 00:00:00,2015-11-26 00:00:00,2018-05-10 20:53:37,7.86,0.0,1847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108706,isoann,797785,14,213,11,20,136,64.372222,Male,Poland:,1997-01-13 00:00:00,2011-10-23 00:00:00,2018-02-25 01:20:27,8.03,2.0,3939
108707,bumcakee,1396285,13,116,5,9,22,41.365972,Female,"nhollywood, california",1998-07-18 00:00:00,2012-07-14 00:00:00,2017-02-14 00:30:00,8.86,24.0,2537
108708,Scarlet95,2478991,6,103,10,8,54,46.827083,Female,Belgium,1995-10-17 00:00:00,2013-04-24 00:00:00,2016-12-18 08:41:00,7.40,1.0,2869
108709,Torasori,3975907,22,239,0,4,176,72.361111,Male,"Latvia, Riga",1998-11-18 00:00:00,2014-07-30 00:00:00,2018-05-24 21:34:46,8.98,47.0,4469


In [37]:
# trying to find out why there are three values for gender. 
UsersDF['gender'].value_counts()


gender
Male          70880
Female        37330
Non-Binary      501
Name: count, dtype: int64

In [41]:
# as a precausionary step, I dropped all na. 
UsersDF = UsersDF.dropna()

In [44]:
UsersDF

,username,user_id,user_watching,user_completed,user_onhold,user_dropped,user_plantowatch,user_days_spent_watching,gender,location,birth_date,join_date,last_online,stats_mean_score,stats_rewatched,stats_episodes
0,karthiga,2255153,3,49,1,0,0,55.091667,Female,"Chennai, India",1990-04-29 00:00:00,2013-03-03 00:00:00,2014-02-04 01:32:00,7.43,0.0,3391
1,Damonashu,37326,45,195,27,25,59,82.574306,Male,"Detroit,Michigan",1991-08-01 00:00:00,2008-02-13 00:00:00,2017-07-10 06:52:54,6.15,6.0,4903
2,bskai,228342,25,414,2,5,11,159.483333,Male,"Nayarit, Mexico",1990-12-14 00:00:00,2009-08-31 00:00:00,2014-05-12 16:35:00,8.27,1.0,9701
3,terune_uzumaki,327311,5,5,0,0,0,11.394444,Female,"Malaysia, Kuantan",1998-08-24 00:00:00,2010-05-10 00:00:00,2012-10-18 19:06:00,9.70,6.0,697
4,Bas_G,5015094,35,114,6,20,175,30.458333,Male,"Nijmegen, Nederland",1999-10-24 00:00:00,2015-11-26 00:00:00,2018-05-10 20:53:37,7.86,0.0,1847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108706,isoann,797785,14,213,11,20,136,64.372222,Male,Poland:,1997-01-13 00:00:00,2011-10-23 00:00:00,2018-02-25 01:20:27,8.03,2.0,3939
108707,bumcakee,1396285,13,116,5,9,22,41.365972,Female,"nhollywood, california",1998-07-18 00:00:00,2012-07-14 00:00:00,2017-02-14 00:30:00,8.86,24.0,2537
108708,Scarlet95,2478991,6,103,10,8,54,46.827083,Female,Belgium,1995-10-17 00:00:00,2013-04-24 00:00:00,2016-12-18 08:41:00,7.40,1.0,2869
108709,Torasori,3975907,22,239,0,4,176,72.361111,Male,"Latvia, Riga",1998-11-18 00:00:00,2014-07-30 00:00:00,2018-05-24 21:34:46,8.98,47.0,4469


# Dealing with Outliers

# Data Formatting

In [48]:
# formatting the gender to be usable in a model
# !NEED TO INCORPERATE IT BACK INTO THE DATAFRAME!
UsersDF_gender_encoded = pd.get_dummies(UsersDF['gender'])

In [49]:
# Checking to make sure it worked.
UsersDF_gender_encoded

,Female,Male,Non-Binary
0,True,False,False
1,False,True,False
2,False,True,False
3,True,False,False
4,False,True,False
...,...,...,...
108706,False,True,False
108707,True,False,False
108708,True,False,False
108709,False,True,False


In [51]:
# incorperated the encoded data back into the original DataFrame 
# ask if I should drop gender?
usersdf_combined = pd.concat([UsersDF, UsersDF_gender_encoded], axis = 1)
usersdf_combined

,username,user_id,user_watching,user_completed,user_onhold,user_dropped,user_plantowatch,user_days_spent_watching,gender,location,birth_date,join_date,last_online,stats_mean_score,stats_rewatched,stats_episodes,Female,Male,Non-Binary
0,karthiga,2255153,3,49,1,0,0,55.091667,Female,"Chennai, India",1990-04-29 00:00:00,2013-03-03 00:00:00,2014-02-04 01:32:00,7.43,0.0,3391,True,False,False
1,Damonashu,37326,45,195,27,25,59,82.574306,Male,"Detroit,Michigan",1991-08-01 00:00:00,2008-02-13 00:00:00,2017-07-10 06:52:54,6.15,6.0,4903,False,True,False
2,bskai,228342,25,414,2,5,11,159.483333,Male,"Nayarit, Mexico",1990-12-14 00:00:00,2009-08-31 00:00:00,2014-05-12 16:35:00,8.27,1.0,9701,False,True,False
3,terune_uzumaki,327311,5,5,0,0,0,11.394444,Female,"Malaysia, Kuantan",1998-08-24 00:00:00,2010-05-10 00:00:00,2012-10-18 19:06:00,9.70,6.0,697,True,False,False
4,Bas_G,5015094,35,114,6,20,175,30.458333,Male,"Nijmegen, Nederland",1999-10-24 00:00:00,2015-11-26 00:00:00,2018-05-10 20:53:37,7.86,0.0,1847,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108706,isoann,797785,14,213,11,20,136,64.372222,Male,Poland:,1997-01-13 00:00:00,2011-10-23 00:00:00,2018-02-25 01:20:27,8.03,2.0,3939,False,True,False
108707,bumcakee,1396285,13,116,5,9,22,41.365972,Female,"nhollywood, california",1998-07-18 00:00:00,2012-07-14 00:00:00,2017-02-14 00:30:00,8.86,24.0,2537,True,False,False
108708,Scarlet95,2478991,6,103,10,8,54,46.827083,Female,Belgium,1995-10-17 00:00:00,2013-04-24 00:00:00,2016-12-18 08:41:00,7.40,1.0,2869,True,False,False
108709,Torasori,3975907,22,239,0,4,176,72.361111,Male,"Latvia, Riga",1998-11-18 00:00:00,2014-07-30 00:00:00,2018-05-24 21:34:46,8.98,47.0,4469,False,True,False


In [53]:
# switching the Dates to DateTime
usersdf_combined['birth_date'] = pd.to_datetime(usersdf_combined['birth_date'])
usersdf_combined['join_date'] = pd.to_datetime(usersdf_combined['join_date'])
usersdf_combined['last_online'] = pd.to_datetime(usersdf_combined['last_online'])
usersdf_combined


,username,user_id,user_watching,user_completed,user_onhold,user_dropped,user_plantowatch,user_days_spent_watching,gender,location,birth_date,join_date,last_online,stats_mean_score,stats_rewatched,stats_episodes,Female,Male,Non-Binary
0,karthiga,2255153,3,49,1,0,0,55.091667,Female,"Chennai, India",1990-04-29,2013-03-03,2014-02-04 01:32:00,7.43,0.0,3391,True,False,False
1,Damonashu,37326,45,195,27,25,59,82.574306,Male,"Detroit,Michigan",1991-08-01,2008-02-13,2017-07-10 06:52:54,6.15,6.0,4903,False,True,False
2,bskai,228342,25,414,2,5,11,159.483333,Male,"Nayarit, Mexico",1990-12-14,2009-08-31,2014-05-12 16:35:00,8.27,1.0,9701,False,True,False
3,terune_uzumaki,327311,5,5,0,0,0,11.394444,Female,"Malaysia, Kuantan",1998-08-24,2010-05-10,2012-10-18 19:06:00,9.70,6.0,697,True,False,False
4,Bas_G,5015094,35,114,6,20,175,30.458333,Male,"Nijmegen, Nederland",1999-10-24,2015-11-26,2018-05-10 20:53:37,7.86,0.0,1847,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108706,isoann,797785,14,213,11,20,136,64.372222,Male,Poland:,1997-01-13,2011-10-23,2018-02-25 01:20:27,8.03,2.0,3939,False,True,False
108707,bumcakee,1396285,13,116,5,9,22,41.365972,Female,"nhollywood, california",1998-07-18,2012-07-14,2017-02-14 00:30:00,8.86,24.0,2537,True,False,False
108708,Scarlet95,2478991,6,103,10,8,54,46.827083,Female,Belgium,1995-10-17,2013-04-24,2016-12-18 08:41:00,7.40,1.0,2869,True,False,False
108709,Torasori,3975907,22,239,0,4,176,72.361111,Male,"Latvia, Riga",1998-11-18,2014-07-30,2018-05-24 21:34:46,8.98,47.0,4469,False,True,False


In [56]:
# drop the gender
usersdf_combined.drop('gender', inplace=True, axis = 1)
usersdf_combined

,username,user_id,user_watching,user_completed,user_onhold,user_dropped,user_plantowatch,user_days_spent_watching,location,birth_date,join_date,last_online,stats_mean_score,stats_rewatched,stats_episodes,Female,Male,Non-Binary
0,karthiga,2255153,3,49,1,0,0,55.091667,"Chennai, India",1990-04-29,2013-03-03,2014-02-04 01:32:00,7.43,0.0,3391,True,False,False
1,Damonashu,37326,45,195,27,25,59,82.574306,"Detroit,Michigan",1991-08-01,2008-02-13,2017-07-10 06:52:54,6.15,6.0,4903,False,True,False
2,bskai,228342,25,414,2,5,11,159.483333,"Nayarit, Mexico",1990-12-14,2009-08-31,2014-05-12 16:35:00,8.27,1.0,9701,False,True,False
3,terune_uzumaki,327311,5,5,0,0,0,11.394444,"Malaysia, Kuantan",1998-08-24,2010-05-10,2012-10-18 19:06:00,9.70,6.0,697,True,False,False
4,Bas_G,5015094,35,114,6,20,175,30.458333,"Nijmegen, Nederland",1999-10-24,2015-11-26,2018-05-10 20:53:37,7.86,0.0,1847,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108706,isoann,797785,14,213,11,20,136,64.372222,Poland:,1997-01-13,2011-10-23,2018-02-25 01:20:27,8.03,2.0,3939,False,True,False
108707,bumcakee,1396285,13,116,5,9,22,41.365972,"nhollywood, california",1998-07-18,2012-07-14,2017-02-14 00:30:00,8.86,24.0,2537,True,False,False
108708,Scarlet95,2478991,6,103,10,8,54,46.827083,Belgium,1995-10-17,2013-04-24,2016-12-18 08:41:00,7.40,1.0,2869,True,False,False
108709,Torasori,3975907,22,239,0,4,176,72.361111,"Latvia, Riga",1998-11-18,2014-07-30,2018-05-24 21:34:46,8.98,47.0,4469,False,True,False


# Handling Duplicates 

In [43]:
# dropped duplicates as a precausionary step. 
UsersDF.drop_duplicates()

,username,user_id,user_watching,user_completed,user_onhold,user_dropped,user_plantowatch,user_days_spent_watching,gender,location,birth_date,join_date,last_online,stats_mean_score,stats_rewatched,stats_episodes
0,karthiga,2255153,3,49,1,0,0,55.091667,Female,"Chennai, India",1990-04-29 00:00:00,2013-03-03 00:00:00,2014-02-04 01:32:00,7.43,0.0,3391
1,Damonashu,37326,45,195,27,25,59,82.574306,Male,"Detroit,Michigan",1991-08-01 00:00:00,2008-02-13 00:00:00,2017-07-10 06:52:54,6.15,6.0,4903
2,bskai,228342,25,414,2,5,11,159.483333,Male,"Nayarit, Mexico",1990-12-14 00:00:00,2009-08-31 00:00:00,2014-05-12 16:35:00,8.27,1.0,9701
3,terune_uzumaki,327311,5,5,0,0,0,11.394444,Female,"Malaysia, Kuantan",1998-08-24 00:00:00,2010-05-10 00:00:00,2012-10-18 19:06:00,9.70,6.0,697
4,Bas_G,5015094,35,114,6,20,175,30.458333,Male,"Nijmegen, Nederland",1999-10-24 00:00:00,2015-11-26 00:00:00,2018-05-10 20:53:37,7.86,0.0,1847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108706,isoann,797785,14,213,11,20,136,64.372222,Male,Poland:,1997-01-13 00:00:00,2011-10-23 00:00:00,2018-02-25 01:20:27,8.03,2.0,3939
108707,bumcakee,1396285,13,116,5,9,22,41.365972,Female,"nhollywood, california",1998-07-18 00:00:00,2012-07-14 00:00:00,2017-02-14 00:30:00,8.86,24.0,2537
108708,Scarlet95,2478991,6,103,10,8,54,46.827083,Female,Belgium,1995-10-17 00:00:00,2013-04-24 00:00:00,2016-12-18 08:41:00,7.40,1.0,2869
108709,Torasori,3975907,22,239,0,4,176,72.361111,Male,"Latvia, Riga",1998-11-18 00:00:00,2014-07-30 00:00:00,2018-05-24 21:34:46,8.98,47.0,4469


# Addressing inconsistent or Erroneous Data


# Feature Engineering

# Data Normalization and Scaling

# Handling Categorical Variables

# Data Validation and Quality Checks


# Documentation